In [ ]:
import os
import numpy as np
import tensorflow as tf
print(tf.__version__)
from scipy.io import loadmat
import h5py
import functools

# %matplotlib widget
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update({'font.size': 16})
mpl.rcParams.update({'font.family': 'serif'})

from utils_data import get_t2f_data

TEST_NAME = 'p4581'
DATA_DIR = '/misc/projects/criticalstress-ldrd/cwj/public/Lab_AE/data_lab_p4581/'

In [ ]:
with open(DATA_DIR+'Mech_Data/p4581_data.txt') as dataFile:
  dataFileLines = dataFile.readlines()

shear_data = []
for i in range(5, len(dataFileLines)):
  fileline = dataFileLines[i].split()
  i_data = [fileline[5], fileline[2], fileline[4]]
  shear_data.append(i_data)
shear_data = np.array(shear_data, dtype=np.float32)

time = shear_data[:,0]
vert_load = shear_data[:,1]
hor_load = shear_data[:,2]

In [ ]:
fig = plt.figure(figsize=(16,4))
ax = plt.gca()
plt.plot(time, hor_load)
plt.xlabel('Time [s]')
plt.ylabel('Normal Stress [MPa]')
plt.grid()

fig = plt.figure(figsize=(16,4))
ax = plt.gca()
plt.plot(time, vert_load)
plt.xlabel('Time [s]')
plt.ylabel('Shear Stress [MPa]')
plt.grid()

fig = plt.figure(figsize=(16,4))
ax = plt.gca()
plt.plot(time, vert_load/hor_load)
plt.xlabel('Time [s]')
plt.ylabel('Shear Stress [MPa]')
plt.grid()

In [ ]:
seg_split = [
    [2025.0, 2275.0, '2mpa_stepup'], 
    [2325.0, 2525.0, '3mpa_stepup'], 
    [2575.0, 2875.0, '4mpa_stepup'], 
    [2925.0, 3175.0, '5mpa_stepup'], 
    [3225.0, 3475.0, '6mpa_stepup'], 
    [3525.0, 3775.0, '7mpa_stepup'], 
    [3825.0, 4075.0, '8mpa_stepup'], 
    [4125.0, 4275.0, '7mpa_stepdown'], 
    [4325.0, 4475.0, '6mpa_stepdown'], 
    [4525.0, 4675.0, '5mpa_stepdown'], 
    [4725.0, 4825.0, '4mpa_stepdown'], 
]

for i_seg_split in seg_split:
    start_time = i_seg_split[0]
    end_time = i_seg_split[1]
    seg_name = i_seg_split[2]
    
    inds_sub = np.where((time>=start_time) & (time<=end_time))

    data_time = np.arange(start_time, end_time, 1.0, dtype=np.float64)
    data_time = np.repeat(data_time, 1000)
    data_time = np.reshape(data_time, [-1,1000])
    data_time = data_time + np.arange(0.0, 1.0, 0.001, dtype=np.float64)
    data_time = np.reshape(data_time, [-1,])

    time_trunc = data_time
    vert_load_trunc = vert_load[inds_sub][:int((end_time-start_time)*1000.0)]
    hor_load_trunc = hor_load[inds_sub][:int((end_time-start_time)*1000.0)]
    frict_coeff_trunc = vert_load_trunc/hor_load_trunc

    time_to_fail, fail_time = get_t2f_data(time_trunc, frict_coeff_trunc, grad_thld=-0.005, drop_thld=0.3, seg_len=int(1.0*1000))

    with h5py.File(DATA_DIR+TEST_NAME+'_mechdata_'+seg_name+'.hdf5', 'w') as f:
      f.create_dataset('time', data = time_trunc)
      f.create_dataset('mu', data = frict_coeff_trunc)
      f.create_dataset('t2f', data = time_to_fail)
      f.create_dataset('tfail', data = fail_time)

    fig = plt.figure(figsize=(24,4))
    ax = plt.gca()
    plt.plot(time_trunc, hor_load_trunc)
    plt.xlabel('Time [s]')
    plt.ylabel('Normal Stress [Mpa]')
    plt.grid()
    plt.title(seg_name+' '+str(np.mean(hor_load_trunc)))
    
    fig = plt.figure(figsize=(24,4))
    ax = plt.gca()
    plt.plot(time_trunc, time_to_fail)
    plt.plot(fail_time, np.zeros_like(fail_time), 'ro')
    plt.xlabel('Time [s]')
    plt.ylabel('T2F [s]')
    plt.grid()

    ax1 = ax.twinx()
    ax1.plot(time_trunc, frict_coeff_trunc, 'k--')
    plt.title(seg_name+' '+str(np.mean(hor_load_trunc)))
    plt.show()
    print('\n\n\n')